# Multipage Tables Scrape Demo

You're often going to encounter data and tables spread across hundreds if not thousands of pages. 

We're going to scrape as a demo a table that runs across several pages on <a href="https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html">this mock website</a>.

To capture your target information into a single CSV file will require the use of many of the foundational skills we've covered, including:

- ```delays```
- ```conditional logic```
- ```while loops```
- ```BeautifulSoup```


And we'll explore a few new functional Python methods today.

## Scraping Strategies

- How do we approach this scrape?
- What pattern do we see?
- How do we capture a table on a single page?
- How do we capture a sequence of tables?
- How we navigate from page 1 to the subsequent pages?

# Let's code!

In [1]:
# import libraries

# from bs4 import BeautifulSoup  ## web scraping
import requests ## request html for a page(s)
import pandas as pd ## pandas to work with data

## Single Table Scrape

In [3]:
## request url website

url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html"
response = requests.get(url)
response.status_code

200

In [7]:
## page content type
type(response.content)

bytes

In [8]:
## page text type
type(response.text)

str

## Since ```page.text``` returns a ```str```, we don't need to use ```BeautifulSoup```.


In [9]:
## use Pandas to read tables on page
df_list = pd.read_html(response.text)
df_list

[                 Animal  Weight(kg)    Type
 0            Blue whale      136000  Marine
 1         Bowhead whale      100000  Marine
 2             Fin whale       70000  Marine
 3  Southern right whale       45000  Marine
 4        Humpback whale       30000  Marine]

In [11]:
## Do we want the first table?
# type(df)
type(df_list[0])

pandas.core.frame.DataFrame

In [13]:
## store it into a copy called animals_df
animals_df = df_list[0]
animals_df

,Animal,Weight(kg),Type
0,Blue whale,136000,Marine
1,Bowhead whale,100000,Marine
2,Fin whale,70000,Marine
3,Southern right whale,45000,Marine
4,Humpback whale,30000,Marine


## But we want to scrape multiple pages
2 ways to build a list of urls that we have to navigate to:

1. Placeholders
2. f-strings

In [ ]:
## Never do this manually
mylinks = ["https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html",
          "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html",
          "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html"]

### 1. Placeholders

In [ ]:
## How is it different?
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"

## Placeholders

<img src="https://raw.githubusercontent.com/sandeepmj/scrape-example-page/master/images/placeholder1.png" style="width:500px;">

## Placeholders

<img src="https://raw.githubusercontent.com/sandeepmj/scrape-example-page/master/images/placeholder2.png" style="width:500px;">

## Placeholders

<img src="https://raw.githubusercontent.com/sandeepmj/scrape-example-page/master/images/placeholder3.png" style="width:500px;">

## Filling the Placeholder

### We use ```.format()``` to fill in values into the ```{}```placeholder

In [14]:
## here's our base url
base_url = "https://www.example{}.html"

In [22]:
## Using a ```for loop```

all_urls = []
for url_number in range(1,7):
    all_urls.append(base_url.format(url_number))
    
all_urls

['https://www.example1.html',
 'https://www.example2.html',
 'https://www.example3.html',
 'https://www.example4.html',
 'https://www.example5.html',
 'https://www.example6.html']

In [23]:
## using list comprehension

all_urls_lc = [base_url.format(url_number) for url_number in range(1, 7)]
all_urls_lc

['https://www.example1.html',
 'https://www.example2.html',
 'https://www.example3.html',
 'https://www.example4.html',
 'https://www.example5.html',
 'https://www.example6.html']

### 2. Using f-strings

In [25]:
## base url of site to scrape
base_url = "https:/www.example"

In [31]:
print(f"{base_url}1.html")

https:/www.example1.html


In [39]:
## Using a ```for loop```
fs_fl = []
for number in range(1,7):
    fs_fl.append(f"{base_url}{number}.html")
    
fs_fl


['https:/www.example1.html',
 'https:/www.example2.html',
 'https:/www.example3.html',
 'https:/www.example4.html',
 'https:/www.example5.html',
 'https:/www.example6.html']

In [41]:
## using list comprehension
fs_lc = [f"{base_url}{number}.html" for number in range(1, 7)]
fs_lc

['https:/www.example1.html',
 'https:/www.example2.html',
 'https:/www.example3.html',
 'https:/www.example4.html',
 'https:/www.example5.html',
 'https:/www.example6.html']

## Back to our scrape

In [42]:
## let's remind ourselves of url variable's value

url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"

## We know we need a placeholder value of upto ```4```
## Let's create a variable called  ```total_pages``` to match number of pages on site.

In [53]:
## total pages to scrape
total_pages = 6
mylinks = [url.format(number) for number in range(0, total_pages)]
mylinks

['https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page4.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page5.html']

# We have a problem...

### We're hitting the server way too fast. We have to add a delay before we proceed.

In [44]:
## Let's import the required libaries to create a delay
from random import randrange ##  allows us to randomize numbers library
import time ## time tracker

In [55]:
## Let's run our code again but with appropriate delay
all_df = [] ##this holds all the scraped df
busted_links = [] ## this holds any links that were broken

counter = 1
for link in mylinks:
    print(f"Scraping {counter} of {len(mylinks)} ")
    counter += 1
    print(f"scraping {link}")
    response = requests.get(link)
    if response.status_code == 200:
        df = pd.read_html(response.text)
        all_df.append(df[0])
    else:
        print(f"{link} returned a busted link with response {response.status_code}")
        busted_links.append(link)
        
    snoozer = randrange(5,12)
    print(f"snoozing for {snoozer} seconds before next scrape")
    time.sleep(snoozer)
    
    
print("All done....")


all_df

Scraping 1 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html returned a busted link with response 404
snoozing for 11 seconds before next scrape
Scraping 2 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html
snoozing for 8 seconds before next scrape
Scraping 3 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html
snoozing for 10 seconds before next scrape
Scraping 4 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html
snoozing for 6 seconds before next scrape
Scraping 5 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page4.html
snoozing for 8 seconds before next scrape
Scraping 6 of 6 
scraping https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page5.html
https://sandeepmj.github.io/scrape-example-page/heaviest-an

[                 Animal  Weight(kg)    Type
 0            Blue whale      136000  Marine
 1         Bowhead whale      100000  Marine
 2             Fin whale       70000  Marine
 3  Southern right whale       45000  Marine
 4        Humpback whale       30000  Marine,
                  Animal  Weight(kg)    Type
 0            Gray whale       28500  Marine
 1  Northern right whale       23000  Marine
 2             Sei whale       20000  Marine
 3         Bryde's whale       16000  Marine
 4  Baird's beaked whale       11380  Marine,
                       Animal  Weight(kg)         Type
 0                Minke whale        7500       Marine
 1  Northern bottlenose whale        6500       Marine
 2     Gervais's beaked whale        5600       Marine
 3           African elephant        4800  Terrestrial
 4               Killer whale        3988       Marine,
                      Animal  Weight(kg)         Type
 0              Hippopotamus        3750  Terrestrial
 1            Asian

In [56]:
busted_links

['https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html',
 'https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page5.html']

In [ ]:
## let's remind ourselves of url variable's value

base_url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page"

In [ ]:
## ## for loop with timer

## Working Around Errors

When you scrape hundreds of pages, there's chance that one of the URLs might be a dud.

We can set up a error control to see what kind of responses we get:

```<Response [200]>``` means website is accessible.

```<Response [404]>``` means broken link or no page on content.

In that case, your whole code might break and you'll have to figure out where it broke.

We can make that easier with conditional logic.

In [ ]:
## CHECK FOR ERROR




In [ ]:
## show broken links


# All in One Step

Because we are using a  ```for loop``` that cycles through each link to do multiple steps on our target data, we need to have it done as one step.



In [ ]:
## Combined url timed nav with table scrape



### What does this list hold?


In [ ]:
## see the list

### Combine all the dataframes into one

In [57]:
## concat here
df = pd.concat(all_df, ignore_index = True)
df

,Animal,Weight(kg),Type
0,Blue whale,136000,Marine
1,Bowhead whale,100000,Marine
2,Fin whale,70000,Marine
3,Southern right whale,45000,Marine
4,Humpback whale,30000,Marine
5,Gray whale,28500,Marine
6,Northern right whale,23000,Marine
7,Sei whale,20000,Marine
8,Bryde's whale,16000,Marine
9,Baird's beaked whale,11380,Marine


In [ ]:
## export to csv

## Let's revisit this in a momment and convert these last couple of steps into a ```function```.

In [66]:
def process_lists(list_name, filename):
    df = pd.concat(list_name, ignore_index = True)
    df.to_csv(filename, encoding = "UTF-8", index = False)
    print(f"{filename} is in your current folder")
    return df

In [67]:
df_animals = process_lists(all_df, "big_animals.csv")

big_animals.csv is in your current folder


In [68]:
df_animals

,Animal,Weight(kg),Type
0,Blue whale,136000,Marine
1,Bowhead whale,100000,Marine
2,Fin whale,70000,Marine
3,Southern right whale,45000,Marine
4,Humpback whale,30000,Marine
5,Gray whale,28500,Marine
6,Northern right whale,23000,Marine
7,Sei whale,20000,Marine
8,Bryde's whale,16000,Marine
9,Baird's beaked whale,11380,Marine
